In [ ]:
import matplotlib
%matplotlib widget

import numpy as np
from split_dataset import SplitDataset
from pathlib import Path
import flammkuchen as fl
import matplotlib.pyplot as plt 

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr
from scipy.signal import detrend 

In [ ]:
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\rf\fixed")
fish_list = list(master.glob("*f[0-9]*"))
fish_dir = fish_list[3]

try:
    with open(next(fish_dir.glob("*metadata.json"))) as i:
        metadata = json.load(i)
    fish_id = metadata['general']['fish_id']
except:
    fish_id = ""
print(fish_dir)
print(fish_id)

In [ ]:
traces = fl.load(fish_dir / "traces.h5")['traces'][:, 0:-2]
fs = 3
t = np.arange(np.shape(traces)[1]) / fs
num_traces, len_rec = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
new_len_rec = int(len_rec/3)

In [ ]:
norm_traces = np.copy(traces)
norm_traces=norm_traces.T# need to transpose it since the functions work like that 
sd=np.nanstd(norm_traces)
mean=np.nanmean(norm_traces)
norm_traces=norm_traces-mean #numerator in the formula for z-score 
norm_traces=norm_traces/sd
norm_traces=norm_traces.T

In [ ]:
#for i in range(num_traces):
#    norm_traces[i] =  norm_traces[i] +np.linspace(0, 10, len_rec)

In [ ]:
norm_traces[np.where(np.isnan(norm_traces))] = 0
#corrected_traces = np.zeros_like(norm_traces)
#for i in range(num_traces):
#    corrected_traces[i] = detrend(norm_traces[i], overwrite_data=False, type='linear')
corrected_traces = detrend(norm_traces, axis=1, overwrite_data=False, type='linear')
fig, ax = plt.subplots(2, 1, figsize=(10,15))
ax[0].imshow(norm_traces, aspect="auto")
ax[1].imshow(corrected_traces,  aspect="auto")

In [ ]:
trial_traces = np.zeros((3, num_traces, new_len_rec))
trial_traces = np.zeros((num_traces, 3, new_len_rec))
trial_traces_corrected = np.zeros((num_traces, 3, new_len_rec))

for i in range(3):
    t1 = i * new_len_rec
    t2 = t1 + new_len_rec
    trial_traces[:, i] = traces[:, t1:t2]
    trial_traces_corrected[:, i] = corrected_traces[:, t1:t2]
avg_traces = np.nanmean(trial_traces_corrected, 1)
print(np.shape(trial_traces))

In [ ]:
n_blocks = 3
dt = 0.33
traces_xr = xr.DataArray(
    data=trial_traces,                               #Adding the data
    dims=['roi', 'block', 't'],                #Defining name of the dimensions
    coords={                                   #Defining values at which each dimension wase valuated
        'roi':np.arange(trial_traces.shape[0]), 
        'block':np.arange(n_blocks),
        't':np.arange(trial_traces.shape[2])*dt
        }
    )
reliability_arr = reliability(np.swapaxes(traces_xr, 0, 2).values)
rel_thresh = threshold_otsu(reliability_arr)
print("Reliability threshold: ", rel_thresh)

rel_thresh_3 = np.round(rel_thresh * 1000)
rel_thresh_3 /=1000

traces_xr_det = xr.DataArray(
    data=trial_traces_corrected,                               #Adding the data
    dims=['roi', 'block', 't'],                #Defining name of the dimensions
    coords={                                   #Defining values at which each dimension wase valuated
        'roi':np.arange(trial_traces_corrected.shape[0]), 
        'block':np.arange(n_blocks),
        't':np.arange(trial_traces_corrected.shape[2])*dt
        }
    )
reliability_arr_det = reliability(np.swapaxes(traces_xr_det, 0, 2).values)
rel_thresh_det = threshold_otsu(reliability_arr_det)
print("Reliability threshold: ", rel_thresh_det)

rel_thresh_3_det = np.round(rel_thresh_det * 1000)
rel_thresh_3_det /=1000
print(np.shape(reliability_arr_det))

In [ ]:
#Visualize
fig, ax = plt.subplots(1, 2, figsize=(8,5))
ax[0].hist(reliability_arr, bins=50, density=True);
ax[0].axvline(rel_thresh, c='red', ls='--')

ax[0].set_xlim([-1,1])
ax[0].set_xlabel('Average correlation between reps')
ax[0].set_ylabel('Density')
ax[0].set_title("Reliability threshold: " + str(rel_thresh_3))
plt.tight_layout()

ax[1].hist(reliability_arr_det, bins=50, density=True);
ax[1].axvline(rel_thresh_det, c='red', ls='--')
ax[1].set_xlim([-1,1])
ax[1].set_xlabel('Average correlation between reps')
ax[1].set_ylabel('Density')
ax[1].set_title("Detrend, Reliability threshold: " + str(rel_thresh_3_det))
plt.tight_layout()

file_name = "reliability index " + fish_id
fig.savefig(str(fish_dir/file_name))

In [ ]:
## removing unreliable traces:

ind_reliable = np.where(reliability_arr_det >= rel_thresh_3_det)
print(np.shape(ind_reliable))
num_reliable_traces = np.shape(ind_reliable)[1]
reliable_traces = norm_traces.T[ind_reliable]
print(np.shape(reliable_traces))

In [ ]:
## save reliable traces:

reliable_traces_save = 

In [ ]:
all_rois = fl.load(merged_rois)
reliable_rois = np,zeros_like(all_rois) 
new_count=1
for i in range(0, num_traces):
    if i is in ind_reliable:
        reliable_rois[np.where(roi_map == (i))] = new_count
        new_count += 1
        
reliable_rois_save = 